# <center><b> Install And Login </center>

In [ ]:
#@title
#@markdown <h3>⬅️ Click this First</h3>
import IPython
from google.colab import output
 
display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button"); 
    btn.click() 
    }
    
  btn = document.getElementById('ok')
  if (btn != null){
    console.log("Click reconnect"); 
    btn.click() 
    }
  }
    
setInterval(ClickConnect,60000)
'''))

In [ ]:
#Run This First
!pip install qiskit ipywidgets
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!pip install folium==0.2.1
 
#@title
#@markdown <h3>⬅️ Click Here to install libtorrent</h3>
 
 
 
!apt install python3-libtorrent
 
import libtorrent as lt
 
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
 
from IPython.display import HTML, clear_output
 
clear_output()
 
print("Server Started Successfully")

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# <center><b> Download </center>

In [ ]:
#@title
#@markdown <h3>Enter files torrent and the path :</h3>
path = "/content/drive/Shared drives/sidc" #@param {type:"string"}
check=[]
 
 
from google.colab import files
 
source = files.upload()
params = {
    "save_path": path,
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))
 
clear_output()
 
print("Started Successfully")

In [ ]:
#@title
#@markdown <h3>Enter the path and Add magnet link :</h3>
path = "/content/drive/Shared drives/Google_Drive/Torrent" #@param {type:"string"}
check=[]
params = {"save_path": path}
 
while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )
 
clear_output()
 
print("Started Successfully")

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to Start Download</h3>
import time
from IPython.display import display
import ipywidgets as widgets
 
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]
 
layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)
 
while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()
 
            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)